Tento notebook bude sluzit pre klasifikaciu nad testovacimi datami. Ktore nasledne ulozime do subor s pridanym stlpcom 'class'.

In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
from sklearn.tree import DecisionTreeClassifier
import category_encoders as ce
import copy
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

In [2]:
merged_data = pd.read_csv('trained_datas.csv',index_col=0)
# merged_data.info()
merged_data = merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = merged_data.select_dtypes(include = ['object'])

encoder = ce.OneHotEncoder()
encoder.fit(categoric)
transformed = encoder.transform(categoric)

merged_data = merged_data.drop(categoric, axis = 1)
merged_data = merged_data.merge(transformed, left_index=True, right_index=True)
merged_data

,education-num,capital-gain,fnlwgt,hours-per-week,capital-loss,mean_glucose,std_glucose,kurtosis_glucose,skewness_glucose,mean_oxygen,...,native-country_37,native-country_38,native-country_39,workclass_1,workclass_2,workclass_3,workclass_4,workclass_5,workclass_6,workclass_7
0,1.595991,-0.307122,1.389199,2.451870,-0.227166,95.351562,41.828816,1.280503,3.351277,14.103679,...,0,0,0,1,0,0,0,0,0,0
1,-0.060364,-0.307122,0.215815,1.190628,-0.227166,111.164062,51.802174,0.267679,-0.141940,2.902174,...,0,0,0,0,1,0,0,0,0,0
2,-0.060364,-0.307122,0.276509,0.774575,-0.227166,14.460938,29.387737,5.600391,34.223119,57.382107,...,0,0,0,0,1,0,0,0,0,0
3,-0.060364,-0.307122,-0.437770,-2.727384,-0.227166,107.796875,41.562709,0.510415,1.672603,1.910535,...,0,0,0,0,1,0,0,0,0,0
4,-0.060364,-0.307122,0.055622,2.451870,-0.227166,111.484375,53.068350,0.459786,-0.116594,1.599498,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,-0.447286,-0.307122,0.674190,-0.049760,-0.227166,103.578125,45.166638,0.552355,0.777646,21.821070,...,0,0,0,0,1,0,0,0,0,0
3929,1.166759,-0.307122,-0.156044,1.190628,-0.227166,27.468750,37.283110,4.430580,20.844460,7.814381,...,0,0,0,0,1,0,0,0,0,0
3930,-0.060364,-0.307122,1.665610,-0.049760,-0.227166,100.859375,32.731324,0.251239,3.491732,4.968227,...,0,0,0,0,1,0,0,0,0,0
3931,-2.176550,-0.307122,0.274628,0.361027,-0.227166,40.398438,33.748458,3.933903,19.294670,96.349498,...,0,0,0,0,1,0,0,0,0,0


In [3]:
cls = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7, \
                       max_features=None, max_leaf_nodes=25, \
                       min_impurity_decrease=0.0, min_impurity_split=None, \
                       min_samples_leaf=4, min_samples_split=2, \
                       min_weight_fraction_leaf=0.0, presort=False, \
                       random_state=0, splitter='random')

X = merged_data[merged_data.columns.difference(['class'])]
y = merged_data['class']
cls = cls.fit(X, y)

In [4]:
graph = Source(export_graphviz(cls, 
                               out_file=None,
                               feature_names=X.columns,
                               class_names=['no diabetes', 'yes diabetes'],
                               filled = True))

# display(SVG(graph.pipe(format='svg')))

# from IPython.display import HTML # toto je tu len pre to aby sa mi obrazok zmestil na obrazovku
# style = "<style>svg{width:70% !important;height:70% !important;}</style>"
# HTML(style)

In [5]:
test_merged_data = pd.read_csv('test_datas.csv',index_col=0)
# val_merged_data.info()
test_merged_data = test_merged_data.drop(['name', 'address', 'date_of_birth'], axis = 1)
categoric = test_merged_data.select_dtypes(include = ['object'])
transformed = encoder.transform(categoric)

test_merged_data = test_merged_data.drop(categoric, axis = 1)
test_merged_data = test_merged_data.merge(transformed, left_index=True, right_index=True)

In [6]:
test_merged_data['class'] = cls.predict(test_merged_data[test_merged_data.columns.difference(['class'])])

In [10]:
test_merged_data.to_csv('final_classification.csv')
test_merged_data['class'].value_counts()

0.0    1051
1.0     261
Name: class, dtype: int64

Celkovy dataset finalnej klasifikacie sme ulozili do suboru 'final_classification.csv'. Tento subor obsahuje 1051 pacientov bez choroby a 261 pacientov s chorobou.

Na klasifikaciu sme pouzili DecisionTree s nastavenim hyperparametrov, pricom tieto parametre sme nastavili podla vysledkov optimalizacie hyperparametrov (obstali ako najlepsie).

Tymito hyperparametrami su:
* class_weight=None
* criterion='entropy'
* max_depth=7
* max_features=None 
* max_leaf_nodes=25
* min_impurity_decrease=0.0 
* min_impurity_split=None
* min_samples_leaf=4 
* min_samples_split=2 
* min_weight_fraction_leaf=0.0 
* presort=False 
* random_state=0
* splitter='random'